# iris example

In [ ]:
from sklearn.datasets import load_iris
from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import pandas as pd
iris = load_iris()

In [ ]:
iris["data"][0].shape

In [ ]:
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data.columns = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]

data["Species"] = iris.target
data

In [ ]:
train_dataset = data.sample(frac=0.8,random_state=0)
test_dataset = data.drop(train_dataset.index)
train_labels = train_dataset.pop('Species')
test_labels = test_dataset.pop('Species')
train_dataset.keys()

In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_labels)
encoded_Y = encoder.transform(train_labels)

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [ ]:
 # define model
def build_model():
    # create model
    sepal_length_input = keras.Input(shape=(1,), name="SepalLength")
    sepal_width_input = keras.Input(shape=(1,), name="SepalWidth")
    petal_length_input = keras.Input(shape=(1,), name="PetalLength")
    petal_width_input = keras.Input(shape=(1,), name="PetalWidth")
    inputs = [sepal_length_input, sepal_width_input, petal_length_input, petal_width_input]
    merged = keras.layers.concatenate(inputs)
    dense1 = Dense(8, activation='relu')(merged)
    output = Dense(3, activation='softmax', name="Species")(dense1)

    # Compile model
    model = keras.Model(inputs=inputs, outputs=[output])
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = build_model()
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

In [ ]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_x = train_dataset.to_dict("series")

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

history = model.fit(train_x, dummy_y, epochs=1000,
                    validation_split = 0.2, verbose=1, callbacks=[early_stop])

In [ ]:
model.save("files/iris_model.h5")

In [ ]:
model.input_names

In [ ]:
model.output_names